In [1]:
from konlpy.tag import Okt
t = Okt()
import re
import gensim
from gensim.models import word2vec
import pandas as pd
import pymongo

In [19]:
df = pd.read_csv('./datas/final2019(rev4).csv', index_col=[0])
df

,Artist,Genre,Lyrics,Title,Year
0,금호동,성인가요,하루의 일을 끝내고 돌아가는거리엔 사람의 물결하늘엔 별이 하나 둘 반짝이면가로등 하...,내일또 만납시다,1964
1,최숙자,성인가요,조기를 담뿍잡아기폭을 올리고온다던 그배는어이하여 아니오나수평선 바라보며그이름 부르면...,눈물의 연평도,1964
2,이미자,성인가요,<1절 임이라 부르리까 당신이라고 부르리까 사랑을 하면서도 사랑을 참고사는 마음으로...,님이라 부르리까,1964
3,이미자,성인가요,헤일 수 없이 수많은 밤을내 가슴 도려내는 아픔에 겨워얼마나 울었던가 동백아가씨그리...,동백아가씨,1964
4,현미,성인가요,그 날 밤 그 자리에둘이서 만났을 때똑같은 그 순간에똑같은 마음이달빛에 젖은 채밤새...,떠날때는 말없이,1964
...,...,...,...,...,...
4253,아이유,록/메탈,‘뭐해?‘라는 두 글자에‘네가 보고 싶어’ 나의 속마음을 담아 우이모티콘 하나하나 ...,Blueming,2019
4254,MC몽,랩/힙합,우선 캐릭터는 분명해야 해요 X 같은 XX철부지같이 굴면Fail 해요 Next 바로...,"인기 (Feat. 송가인, 챈슬러)",2019
4255,ITZY (있지),댄스,Hey hey hey yass whoo Beep beep Hey hey hey he...,ICY,2019
4256,Lil tachi,랩/힙합,멋이 다 흘러흘러서 넘쳐멀어 버렸어배로 막 벌어멋이 다 흘러멋이 다 흘러배로 막 벌...,눈 (Prod. 기리보이),2019


In [2]:
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

In [21]:
yrange = df[(df['Year'] < 2013) & (df['Year'] >= 2009)].reset_index(drop=True)
lyrics = yrange['Lyrics'].apply(cleanse)
lyrics

0      Uh Huh Listen Boy My First Love Story My Angel...
1      hey playboyit s about time and your time s upI...
2      Abracadabra이러다 미쳐내가 여리여리 착하던 그런 내가너 때문에 돌아내가 독...
3      A yo finallyIs this what you been waitin  forb...
4      Again and again and again and againAgain and a...
                             ...                        
395    나는 너에게 사랑을 구걸하지 않았어진심을 원했어마지막으로 널 봤던 날도널 원하지 않...
396    그대 울지 말아요 슬퍼하지 말아요 사랑은 다 그런 거래요 마음 주면 줄수록 가슴 아...
397    Ha Ha Yeah BBC follow meBounce like this eh eh...
398    나 혼자 미치도록 너만을 미치도록 널 사랑하고 사랑한 그게 잘못이라면 아니야 아니야...
399    주르르루 주르르루 빗소리에현관으로 나갔는데 우산이 없어아니 벌써 어두컴컴 축축한 하...
Name: Lyrics, Length: 400, dtype: object

In [22]:
lyrics = lyrics.tolist()
lyrics

['Uh Huh Listen Boy My First Love Story My Angel and My GirlMy SunshineUh Uh Let s go너무 너무 멋져 눈이 눈이 부셔숨을 못 쉬겠어 떨리는 GirlGee Gee Gee Gee Baby Baby BabyGee Gee Gee Gee Baby Baby Baby너무 부끄러워 쳐다 볼 수 없어사랑에 빠져서 수줍은 GirlGee Gee Gee Gee Baby Baby BabyGee Gee Gee GeeBe Be Be Be Be Be어떻게 하죠어떡 어떡하죠떨리는 나는떨리는 나는요두근 두근 두근 두근두근 두근거려밤엔 잠도 못 이루죠나는 나는 바본가봐요그대 그대밖에 모르는 바보그래요 그댈 보는 난너무 반짝 반짝 눈이 부셔No No No No No너무 깜짝 깜짝 놀란 나는Oh Oh Oh Oh Oh너무 짜릿 짜릿 몸이 떨려Gee Gee Gee Gee GeeOh 젖은 눈빛 Oh Yeah좋은 향기 Oh Yeah Yeah Yeah너무 너무 예뻐 맘이 너무 예뻐첫 눈에 반했어 꼭 짚은 GirlGee Gee Gee Gee Baby Baby BabyGee Gee Gee Gee Baby Baby Baby너무나 뜨거워 만질 수가 없어사랑에 타버려 후끈한 GirlGee Gee Gee Gee Baby Baby BabyGee Gee Gee GeeBe Be Be Be Be Be어쩌면 좋아어쩌면 좋아요수줍은 나는수줍은 나는요몰라 몰라 몰라 몰라몰라 몰라 하며매일 그대만 그리죠친한 친구들은 말하죠정말 너는 정말 못 말려바보 하지만 그댈 보는 난너무 반짝 반짝 눈이 부셔No No No No No너무 깜짝 깜짝 놀란 나는Oh Oh Oh Oh Oh너무 짜릿 짜릿 몸이 떨려Gee Gee Gee Gee GeeOh 젖은 눈빛 Oh Yeah좋은 향기 Oh Yeah Yeah Yeah말도 못했는 걸너무 부끄러워 하는 난용기가 없는 걸까어떡해야 좋은걸까두근 두근 맘 졸이며바라보고 있는 난너무 반짝 반짝 눈이 부셔No No No No No너무 깜짝 깜짝 놀란

In [8]:
results = []
for i in lyrics:
    result = t.pos(i, norm=True, stem=True)
        
    word = []
    for a in result:
        if not a[1] in ["Josa", "Determiner", "Exclamation", "Suffix"]:
            word.append(a[0])
    words = (" ".join(word)).strip()
    results.append(words)

In [10]:
data_file = 'lyrics.data'
with open(data_file, 'w', encoding='utf-8') as fp:
    fp.write("\n".join(results))
data = word2vec.LineSentence(data_file)
model = word2vec.Word2Vec(data, size=200, window=10, hs=1, min_count=2, sg=1)
model.save('lyrics.model')
model = word2vec.Word2Vec.load("lyrics.model")
model

In [11]:
song = model.wv.most_similar('사랑', topn=10)
song

[('해이', 0.6840728521347046),
 ('하다', 0.681125819683075),
 ('지라', 0.6800288558006287),
 ('비록', 0.660918653011322),
 ('익숙해지다', 0.6589189171791077),
 ('머물다', 0.6570615768432617),
 ('분하다', 0.6566281318664551),
 ('쥐', 0.6516861319541931),
 ('해내다', 0.6512840986251831),
 ('난정', 0.6507854461669922)]

In [17]:
keyword = []
for s in song:
    keyword.append(s[0])
relist = yrange[yrange['Lyrics'].str.contains('|'.join(keyword))]
len(relist)

57

In [16]:
relist.sample(n=10)

,Artist,Genre,Lyrics,Title,Year
29,김경록 (V.O.S),발라드,차라리 만나지 말 걸 그랬나봐이게 뭐야 결국 이렇게 될거면서우리 그만 헤어지자우리 ...,이젠 남이야 (Feat. Baby-J Of Jewelry),2009
324,버벌진트,랩/힙합,"girl,you knowyou deserve better충분히 예뻐그런 남자 때문에...",충분히 예뻐 (feat. 산체스 Of 팬텀),2012
354,비스트,댄스,그대는 어떤가요 가끔은 나를 생각한 적 있나요이 긴 밤에 다시 눈을 감고그댈 생각하...,Midnight (별 헤는 밤),2012
273,써니힐,댄스,Midnight Circus Riddle me 어머 Midnight Circus R...,Midnight Circus,2011
202,2NE1,댄스,내가 제일 잘 나가내가 제일 잘 나가내가 제일 잘 나가내가 제일 잘 나가제 제 제일...,내가 제일 잘 나가,2011
140,서영은,발라드,사랑해 사랑해 사랑해이젠 다 가슴에 묻어도한 방울 한 방울 한 방울눈물로 새나와사랑...,이 거지같은 말 (With 정엽 Of 브라운 아이드 소울),2010
211,미쓰에이,댄스,"Good bye, baby good bye뒤돌아서 그대로 앞으로 가면 돼아무런 말도...",Good-bye Baby,2011
308,아이유,댄스,Monday Better day처음처럼 설레이는 그런 날Sunday Better d...,하루 끝,2012
399,버스커 버스커,록/메탈,주르르루 주르르루 빗소리에현관으로 나갔는데 우산이 없어아니 벌써 어두컴컴 축축한 하...,소나기 (주르르루),2012
31,백지영,발라드,우리 서로 사랑했는데우리 이제 헤어지네요같은 하늘 다른 곳에 있어도부디 나를 잊지 ...,잊지 말아요,2009
